In [1]:
!pip install -qU langchain langchain-google-genai google-generativeai==0.8.5 ddgs wikipedia

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 77.5 MB/s eta 0:00:00


In [3]:
!pip install -qU langchain-community

In [14]:
import os
import math
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.agent_toolkits.load_tools import load_tools
from langgraph.prebuilt import create_react_agent # Modern non-deprecated module

# 1. Setup Environment
os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY_DEFAULT")

# 2. Initialize Model
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

# 3. Define Tools using the modern @tool decorator
@tool
def calculator(expression: str) -> str:
    """Useful for mathematical calculations. Input is a math expression like '2+2'."""
    try:
        # Replace ^ with ** for python compatibility
        clean_expr = expression.replace("^", "**")
        allowed_names = {k: v for k, v in math.__dict__.items() if not k.startswith("__")}
        result = eval(clean_expr, {"__builtins__": {}}, allowed_names)
        return f"The result is: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Initialize standard community tools
search = DuckDuckGoSearchRun()
tools = [calculator, search]

# 4. Create the Modern Agent
# This replaces initialize_agent and AgentExecutor
agent_executor = create_react_agent(llm, tools)

# 5. Execution Logic (using .invoke)
def chat_with_agent():
    print("\nModern Agent Mode (Type 'exit' to quit)")
    while True:
        user_input = input("\nYou: ").strip()
        if user_input.lower() in ['exit', 'quit']: break

        # Modern agents use .invoke() and return a state dictionary
        response = agent_executor.invoke({"messages": [("user", user_input)]})

        # The last message in the state is the agent's response
        print(f"Agent: {response['messages'][-1].content}")

if __name__ == "__main__":
    chat_with_agent()


/tmp/ipython-input-499994645.py:35: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(llm, tools)



Modern Agent Mode (Type 'exit' to quit)

You:  Multi-Tool Agen
Agent: [{'type': 'text', 'text': 'I can help with many things! What would you like to do or know?', 'extras': {'signature': 'CvkBAb4+9vv264PEB+6E4zwuGJqoy/KwRWAcqNrhcb+t2vGBnq/TzszeCE/C+T+FqOUh9wQIqZCC/ZCV7Jiqt8/7k5K6sv/GoI2+0kiVRpefVD1ifNsIyCX52Jdqw1ZLUL/nGxPLys59gbIvaipCEeA+mqz/PWGQWr00tdBjuatpmBYeLVLa8ceT/oOQE2iEZKKUOiLZ+sft0SWbaU3oliS/i9y9yq7C26CaDbI1apG6coOLL5Cgj4D2pS7K084ilFwqAgJnCgA4TPvEDOEzsC8ex9RH+Lny/jWIo+pLiatnJ+0lA+cheoHFWEu0TVKuaTVOoduGYM3SLXTJ'}}]

You: What is 15 * 23 + 45?
Agent: The result is 390.

You: Who is the current president of France and what is the square root of 144?


Agent: [{'type': 'text', 'text': 'The current president of France is Emmanuel Macron and the square root of 144 is 12.', 'extras': {'signature': 'CuACAb4+9vv233cfAMzM88XweCZopKgMm7O5oz3wRobvfosyLFFns+3NqAF5VW30REbbx/MaMRkC2U6QYTNROuaNTMaQ+ZnMJiAV/9NiR9TQlJgMXwHCuvlpmgHc+Nvk9rv7c8O0HYhtm+RGwOvZq5ijbmkgPDQGMXPDk/WdhJIXjS8sZKE5GmWgngfqX1dvWTZwJ5gIsrDm5O/v1BqwRq2tG+yhyZP4nedvawpZl5zg+IFvcin5u5tAKGtYq60Qebg+Ba8Eokrdi4X/JzgufNxczXghxeJrPMmkuBnT2rBgDnuxezVxX+Fii1Gd4Kz6hMxnrTOhtbMBfo1woiyM+CR1zLGQHX22DbcTiGTrOn2vQqkyfJ47A9lnEwjcupFCmQT0FUNXIZ2Ti4qrCaIRJ8/XIPuCJxgLCaEgd1+f7itKcNHVtZ7TQDajdoBCvkSrGVzhDxRxGFZpRF1D2RSTrqLMuQ=='}}]

You: Search for the latest news about Agentic AI developments in 2026
Agent: [{'type': 'text', 'text': 'I cannot search for news in the future. I can only access information up to the current date.', 'extras': {'signature': 'CrMHAb4+9vsvdBclbYr1GTFaKw4X+SN4UI2HSrYDVEkV95ctqmF842k2gQxZw0L0dqkPQTeKDr2FknTYM0zGw4iX0VZaLQdNQNZwC29ekzy+4FiyrHPvT1nkgr4CYLBHLX/1jOgNGnhof35UHc4